# Extract

In [6]:
import json
from flows.etl.utils.functions import DB_MinioClient
import pandas as pd
import io

FILE_NAME = "pistacho.json"
FILE_PATH = "ERP/unknown"

# Connect to MinIO
minio_client = DB_MinioClient().connect()
# Get pistacho.json from MinIO and deserialize it
pistacho_json_data = minio_client.get_object(
    "trusted-zone", f"{FILE_PATH}/{FILE_NAME}").read().decode("utf-8")
pistachio_json = json.loads(pistacho_json_data)

# Transform

In [7]:
# Convert json to json compatible with model schema
from flows.etl.manual.pistachos_json.dto.pistacho_dto import Pistachio


farm_input = Pistachio.from_dict(pistachio_json)

farm = {
    "healthAdvisor":  map(lambda ha:
                          {
                              "hadId": ha.HAd_Id,
                              "name": ha.Name,
                              "NIFCode": ha.NIF_Code,
                              "ROPOCode": ha.ROPO_Code,
                              "carnetType": ha.Carnet_Type
                          },
                          farm_input.Farm.HeatlhAdvisor),
    "healthApplicator": map(lambda ha:
                            # TODO: convert to bson datetime
                            {
                                "description": ha.Description,
                                "ROMACode": ha.ROMA_Code,
                                "adquisitionDate": ha.Adquisition_Date,
                                "lastInspectionDate": ha.Last_Inspection_Date,
                                "haId": ha.HA_Id
                            },
                            farm_input.HealthApplicator),
    "farmHolder": {
        "name": farm_input.Farm.FarmHolder.Name,
        "idCode": map(lambda idc:
                      {
                          "type": idc.Type,
                          "code": idc.Code
                      },
                      farm_input.Farm.FarmHolder.Identifier_Code),
        "address": {
            "street": farm_input.Farm.FarmHolder.Address.Street,
            "village": farm_input.Farm.FarmHolder.Address.Village,
            "province": farm_input.Farm.FarmHolder.Address.Province,
            "ZIP": farm_input.Farm.FarmHolder.Address.ZIP
        },
        "phone": farm_input.Farm.FarmHolder.Phone,
        "email": farm_input.Farm.FarmHolder.Email
    },
}



excel_almendros_cercanos
excel_almendros_cercanos
excel_almendros_cercanos
excel_almendros_cercanos
excel_almendros_cercanos
excel_almendros_cercanos
excel_almendros_cercanos


KeyError: 'x-amz-meta-type'

# Load

In [ ]:
# Connect to MongoDB
import os
from pymongo import MongoClient


MONGODB_HOST = os.environ.get("PREFECT_MONGODB_HOST")
MONGODB_DB = os.environ.get("PREFECT_MONGODB_DB")
mongo_client = MongoClient(MONGODB_HOST)
# Get MongoDB database and collection
db = mongo_client[MONGODB_DB]
collection = db["Farm"]
# Store processed data in MongoDB
collection.update_one({
    "farmId": farm["Farm"]["Farm_Id"],
}, {"$set": farm}, upsert=True)
